In [4]:
from explainers.gnninterpreter import * 


In [5]:
import torch
from torch import nn

In [6]:
datasetLoader = torch.load('model/test_loader_red_ratio.pt')
dataset = datasetLoader.dataset

In [7]:
len(dataset)

8000

In [8]:
model = GCNClassifier(node_features=3,
                      num_classes=2,
                      hidden_channels=32)

model.load_state_dict(torch.load('model/model_red_class.pt'))

<All keys matched successfully>

In [9]:
def mean_embeddingsf(dataloader, model, batch_size=32):
    embeds = [[] for _ in range(2)]
    model.eval()
    for batch in dataloader:
        for i, e in enumerate(model(batch)['embeds']):
            embeds[batch.y[i].item()].append(e)
    return [torch.stack(e, dim=0).mean(axis=0) for e in embeds]

In [10]:
mean_embeddings = mean_embeddingsf(datasetLoader, model)

In [11]:
trainer = {}
sampler ={}

In [12]:
cls_idx = 1
trainer[cls_idx] = Trainer(
    sampler=(s := GraphSampler(
        max_nodes=20,
        num_node_cls=2,
        temperature=0.15,
        learn_node_feat=True
    )),
    discriminator=model,
    criterion=WeightedCriterion([
        dict(key="logits", criterion=ClassScoreCriterion(class_idx=cls_idx, mode='maximize'), weight=1),
        dict(key="embeds", criterion=EmbeddingCriterion(target_embedding=mean_embeddings[cls_idx]), weight=10),
        dict(key="logits", criterion=MeanPenalty(), weight=0),
        dict(key="omega", criterion=NormPenalty(order=1), weight=1),
        dict(key="omega", criterion=NormPenalty(order=2), weight=1),
        dict(key="xi", criterion=NormPenalty(order=1), weight=0),
        dict(key="xi", criterion=NormPenalty(order=2), weight=0),
        # dict(key="eta", criterion=NormPenalty(order=1), weight=0),
        # dict(key="eta", criterion=NormPenalty(order=2), weight=0),
        dict(key="theta_pairs", criterion=KLDivergencePenalty(binary=True), weight=0),
    ]),
    optimizer=(o := torch.optim.SGD(s.parameters(), lr=1)),
    scheduler=torch.optim.lr_scheduler.ExponentialLR(o, gamma=1),
    dataset=dataset,
    budget_penalty=BudgetPenalty(budget=10, order=2, beta=1),
    target_probs={cls_idx: (0.9, 1)},
    k_samples=16
)

In [13]:
trainer[1].train(2000)

RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment